In [1]:
#nbi:hide_in
#nbi:hide_out
!pip3 install matplotlib
!pip3 install keras
!pip3 install spectral
!pip3 install seaborn
!pip3 install scikit-image
!pip3 install tensorflow
!pip3 install sklearn

     |████████████████████████████████| 10.3 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 82.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 83.7 MB/s eta 0:00:01
     |████████████████████████████████| 636 kB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.1 MB 36.8 MB/s eta 0:00:01
     |████████████████████████████████| 27.4 MB 46.5 MB/s eta 0:00:01
     |████████████████████████████████| 212 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 285 kB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 29.2 MB 4.5 MB/s eta 0:00:01     |████████████████████████████▍   | 25.9 MB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 71.1 MB/s eta 0:00:01
     |████████████████████████████████| 163 kB 52.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 86.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 48.7 MB/s eta 0:00:01
     |███████████████████████████████

     |████████████████████████████████| 57 kB 9.3 MB/s  eta 0:00:01
     |████████████████████████████████| 1.0 MB 56.5 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 45.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 42.0 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 72.7 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 62.7 MB/s eta 0:00:01
     |████████████████████████████████| 298 kB 91.3 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 9.4 MB/s  eta 0:00:01
     |████████████████████████████████| 136 kB 79.1 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 49.2 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 9.2 MB/s  eta 0:00:01
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=e2191b7db743d0690c06232cc33d1aa8855295d2d8226def5d58871c58eef62d
  Stored in directory: /home/jovyan/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de

In [2]:
#nbi:hide_in
%matplotlib inline
import spectral as sp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from spectrum import rank_normalizer,scale_max_min,scale_white_dark,plot_spectra,zoom
from spectrum_ml import cluster_pixels_kmeans,detect_aphid,blob_detector
from ipywidgets import interact, interactive, widgets
from IPython.display import display

In [2]:
# Create text widget for output
output_slider_variable = widgets.Text()

In [3]:
# Define function to bind value of the input to the output variable 
def f(x):
    output_slider_variable.value = str(x)



In [4]:
# Create input slider with default value = 10    
interact(f, x='adresse');



interactive(children=(Text(value='adresse', description='x'), Output()), _dom_classes=('widget-interact',))

In [5]:
# Display output variable in text box
output_slider_variable



Text(value='adresse')

In [6]:
# Create and output new int variable with value of slider
new_variable = str(output_slider_variable.value)
print(new_variable)



adresse


In [6]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
# Create text widget for output
output_slider_variable1 = widgets.Text()
output_slider_variable2 = widgets.Text()
output_slider_variable3 = widgets.Text()

In [4]:
options1 = {
    'title': 'Exp_name',
}

options2 = {
    'title': 'dossier courant',
}

options3 = {
    'title': 'Fichier csv',
}

In [5]:
# Define function to bind value of the input to the output variable 
def name(x):
    output_slider_variable1.value = str(x)
    
def dossier_courant(x):
    output_slider_variable2.value = str(x)
    
def fichier_csv(x):
    output_slider_variable3.value = str(x)

In [8]:
def adresse(x):
    return x

In [9]:
exp_name = interactive(adresse, x = "VL1grF", options = options1)

In [11]:
display(exp_name, options = options1)

interactive(children=(Text(value='VL1grF', description='x'), Output()), _dom_classes=('widget-interact',))

In [12]:
current_dir = interactive(adresse, x = "/home/plavainne/Documents/BIM/06022020/06022020/")

In [13]:
display(current_dir)

interactive(children=(Text(value='/home/plavainne/Documents/BIM/06022020/06022020/', description='x'), Output(…

In [14]:
fichier_csv = interactive(adresse, x = "wave_lenghts.csv")

In [15]:
display(fichier_csv)

interactive(children=(Text(value='wave_lenghts.csv', description='x'), Output()), _dom_classes=('widget-intera…

In [16]:
wave_lengths = pd.read_csv(fichier_csv.result,index_col=0,header=None)[1]

FileNotFoundError: [Errno 2] No such file or directory: 'wave_lenghts.csv'

In [ ]:
image_path = current_dir.result+exp_name.result+'/capture/'+exp_name.result+'.hdr'
dark_ref_path = current_dir.result+exp_name.result+'/capture/DARKREF_'+exp_name.result+'.hdr'
white_ref_path = current_dir.result+exp_name.result+'/capture/WHITEREF_'+exp_name.result+'.hdr'